In [1]:
import torch
from utils import DataLoader, evaluate_model
from models import LinearRegressionModel
from matplotlib import pyplot as plt

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
dl = DataLoader(
    benchmark="benchmark_2",
    split=None,
    embedding="bert_cls", 
    max_len=16,
    include_axes=True,
    shuffle_axes=False,  
    normalization="zscore"
)

INFO:utils.dataloader:Initializing DataLoader...
INFO:utils.dataloader:Using device: cuda
INFO:utils.dataloader:Loading dataset for benchmark benchmark_2 with split None
INFO:utils.dataloader:No split specified; loading all splits (train, validation, test).
INFO:utils.dataloader:Dataset loaded successfully.
INFO:utils.dataloader:Initializing embedding for method bert_cls...
INFO:utils.dataloader:Initialized BERT model and tokenizer.
INFO:utils.dataloader:Embedding resources initialized.


In [4]:
data = dl.preprocess()
X_train, y_train = data["train"]
X_valid, y_valid = data["validation"]
X_test, y_test = data["test"]

INFO:utils.dataloader:Starting preprocessing of dataset(s)...
INFO:utils.dataloader:Processing split: train with 365 samples...
INFO:utils.dataloader:Processing a single dataset split...
INFO:utils.dataloader:Combined feature vector shape: torch.Size([365, 6912])
INFO:utils.dataloader:Extracting and normalizing target values...
INFO:utils.dataloader:Normalizing target values...
INFO:utils.dataloader:Z-score parameters: mean=[4.64040685e+01 9.55949315e+00 1.10821918e-02], std=[13.53428538  2.54048638  1.18524837]
INFO:utils.dataloader:Processing split: validation with 104 samples...
INFO:utils.dataloader:Processing a single dataset split...
INFO:utils.dataloader:Combined feature vector shape: torch.Size([104, 6912])
INFO:utils.dataloader:Extracting and normalizing target values...
INFO:utils.dataloader:Normalizing target values...
INFO:utils.dataloader:Z-score parameters: mean=[43.97461538  9.55625     0.07490385], std=[12.19832396  2.78474802  1.2170377 ]
INFO:utils.dataloader:Processi

In [5]:
print("Shapes:", X_train.shape, y_train.shape)

if isinstance(X_train, torch.Tensor):
    X_train = X_train.cpu().numpy()
if isinstance(X_valid, torch.Tensor):
    X_valid = X_valid.cpu().numpy()
if isinstance(X_test, torch.Tensor):
    X_test = X_test.cpu().numpy()

Shapes: torch.Size([365, 6912]) (365, 3)


In [6]:
grid_params = {
    "estimator__fit_intercept": [True, False]
}

In [7]:
lr_model = LinearRegressionModel()
lr_model.build_model()

INFO:models.linear_regression:Linear Regression model built (wrapped in MultiOutputRegressor).


In [8]:
best_model, best_params = lr_model.grid_search(X_train, y_train, X_valid, y_valid, param_grid=grid_params, cv=5)

print("Best Hyperparameters:", best_params)

INFO:models.linear_regression:Starting grid search for Linear Regression...


Fitting 5 folds for each of 2 candidates, totalling 10 fits


INFO:models.linear_regression:Grid search complete.
INFO:models.linear_regression:Best Hyperparameters: {'estimator__fit_intercept': False}
INFO:models.linear_regression:Validation MSE for best model: 1.1056


Best Hyperparameters: {'estimator__fit_intercept': False}


In [9]:
test_preds = best_model.model.predict(X_test)
mse, mae, r2, nll, crps = evaluate_model(y_test, test_preds)
print("Test Performance:")
print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}, NLL: {nll:.4f}, CRPS: {crps:.4f}")

Test Performance:
MSE: 1.1009, MAE: 0.8361, R2: -0.1009, NLL: 1.4670, CRPS: 0.8361


In [10]:
save_path = "baseline_weights/benchmark_2/linear_regression.pkl"
best_model.save(save_path)

INFO:models.linear_regression:Linear Regression model saved to baseline_weights/benchmark_2/linear_regression.pkl
